In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
happy_file = "2019_happiness_data.csv"
happy_data_df = pd.read_csv(happy_file)
happy_data_df.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [3]:
govt_file = "govt_spend_to_gdp.csv"
govt_data_df = pd.read_csv(govt_file)
govt_data_df.head()

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2017,2018,2019,2020,2021,2022,2023,2024,Estimates Start After
0,Afghanistan,General government total expenditure,Percent of GDP,NaN,See notes for: General government total expen...,25.878,27.028,28.212,30.310,31.407,32.584,34.159,34.771,2018.0
1,Albania,General government total expenditure,Percent of GDP,NaN,See notes for: General government total expen...,29.137,29.265,29.668,29.287,29.253,28.896,28.560,28.579,2018.0
2,Algeria,General government total expenditure,Percent of GDP,NaN,See notes for: General government total expen...,39.205,38.138,38.722,34.286,32.740,30.125,27.547,27.471,2018.0
3,Angola,General government total expenditure,Percent of GDP,NaN,See notes for: General government total expen...,23.806,19.695,19.205,20.800,20.685,20.411,19.910,19.548,2018.0
4,Antigua and Barbuda,General government total expenditure,Percent of GDP,NaN,See notes for: General government total expen...,23.243,24.909,24.513,23.765,23.868,24.114,23.943,24.181,2018.0


In [11]:
new_happy_data_df = happy_data_df[['Overall rank', 'Country or region','Score', 'GDP per capita', 'Social support', 'Healthy life expectancy']].copy()
new_happy_data_df = new_happy_data_df.rename(columns={'Country or region':'country', 'Score':'score', 'Overall rank':'overall_rank', 'GDP per capita':'gdp_per_capita','Social support':'social_support', 'Healthy life expectancy':'healthy_life_expectancy'})
new_happy_data_df.head()

,overall_rank,country,score,gdp_per_capita,social_support,healthy_life_expectancy
0,1,Finland,7.769,1.340,1.587,0.986
1,2,Denmark,7.600,1.383,1.573,0.996
2,3,Norway,7.554,1.488,1.582,1.028
3,4,Iceland,7.494,1.380,1.624,1.026
4,5,Netherlands,7.488,1.396,1.522,0.999


In [23]:
new_govt_data_df = govt_data_df[['Country', 'Units', '2019']].copy()
new_govt_data_df = new_govt_data_df.rename(columns={'Country':'country','Units':'units' })
new_govt_data_df = new_govt_data_df.dropna(how='any')
new_govt_data_df.head()

,country,units,2019
0,Afghanistan,Percent of GDP,28.212
1,Albania,Percent of GDP,29.668
2,Algeria,Percent of GDP,38.722
3,Angola,Percent of GDP,19.205
4,Antigua and Barbuda,Percent of GDP,24.513


In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/country_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['happy_data', 'govt_data']

In [27]:
new_happy_data_df.to_sql(name='happy_data', con=engine, if_exists='append', index=False)

In [28]:
new_govt_data_df.to_sql(name='govt_data', con=engine, if_exists='append', index=False)

In [29]:
pd.read_sql_query('select * from happy_data', con=engine).head()

,overall_rank,country,score,gdp_per_capita,social_support,healthy_life_expectancy
0,1,Finland,7.769,1.340,1.587,0.986
1,2,Denmark,7.600,1.383,1.573,0.996
2,3,Norway,7.554,1.488,1.582,1.028
3,4,Iceland,7.494,1.380,1.624,1.026
4,5,Netherlands,7.488,1.396,1.522,0.999


In [30]:
pd.read_sql_query('select * from govt_data', con=engine).head()

,country,units,2019
0,Afghanistan,Percent of GDP,28.212
1,Albania,Percent of GDP,29.668
2,Algeria,Percent of GDP,38.722
3,Angola,Percent of GDP,19.205
4,Antigua and Barbuda,Percent of GDP,24.513
